In [49]:
from collections import deque
from math import sqrt
import numpy as np

class RunningMean(object):

    def __init__(self, xs=[]):
        self.q = deque(xs)
        self.N = len(xs)
        self.mean = np.mean(xs) if self.N!=0 else 0.0

    def push(self, x):
        self.q.append(x)
        self.mean = 1 / (self.N + 1) * (self.N * self.mean + x)
        self.N += 1

    def pop(self):
        if self.N == 0:
            raise Exception("popping from an empty list")
        x_pop = self.q.popleft()
        self.mean = self.N / (self.N - 1) * (self.mean - x_pop / self.N) if self.N!=1 else 0.0
        self.N -= 1
        return x_pop

    def pushpop(self, x):
        if self.N == 0:
            raise Exception("pushpopping from an empty list")
        x_pop = self.q.popleft()
        self.q.append(x)
        self.mean = self.mean + (x - x_pop) / self.N
        return x_pop


class RunningSimpleStats(object):

    def _cov(xs,ys):
        return np.cov(xs,ys, bias=True)[0][1]

    def __init__(self, xs=[], ys=[]):

        if len(xs) != len(ys):
            raise Exception("features are not of the same length")
        
        self.xs = RunningMean(xs)
        self.ys = RunningMean(ys)
        self.N = len(xs)

        xs = np.array(xs)
        ys = np.array(ys)
        self._x2s = RunningMean(xs**2)
        self._y2s = RunningMean(ys**2)
        self._xys = RunningMean(xs*ys)

        self.x_var = np.var(xs) if self.N!=0 else 0.0
        self.y_var = np.var(ys) if self.N!=0 else 0.0
        self.xy_cov = RunningSimpleStats._cov(xs,ys) if self.N!=0 else 0.0

    def _update_var_cov(self):
        # Update the variances.
        self.x_var = self._x2s.mean - self.xs.mean**2
        self.y_var = self._y2s.mean - self.ys.mean**2
        # Update Covariance.
        self.xy_cov = self._xys.mean - self.xs.mean*self.ys.mean

    def push(self, x, y):
        self.xs.push(x)
        self.ys.push(y)
        self._x2s.push(x*x)
        self._y2s.push(y*y)
        self._xys.push(x*y)
        self.N += 1
        RunningSimpleStats._update_var_cov()

    def pop(self):
        if self.N == 0:
            raise Exception("popping from an empty list")
        self.xs.pop()
        self.ys.pop()
        self._x2s.pop()
        self._y2s.pop()
        self._xys.pop()
        self.N -= 1
        RunningSimpleStats._update_var_cov()

    def pushpop(self, x, y):
        if self.N == 0:
            raise Exception("pushpopping from an empty list")
        self.xs.pushpop(x)
        self.ys.pushpop(y)
        self._x2s.pushpop(x*x)
        self._y2s.pushpop(y*y)
        self._xys.pushpop(x*y)
        RunningSimpleStats._update_var_cov()


In [52]:
from pprint import pprint

rss = RunningSimpleStats()
rss.push(1,1)
rss.push(2,2)
rss.push(3,4)
rss.pushpop(3,4)
rss.pushpop(1,1)
rss.pushpop(2,2)
rss.pushpop(3,4)
pprint(vars(rss))
pprint(vars(rss.xs))
pprint(vars(rss.ys))
pprint(vars(rss._x2s))
pprint(vars(rss._y2s))
pprint(vars(rss._xys))


{'N': 3,
 '_x2s': <__main__.RunningMean object at 0x11ec1aa10>,
 '_xys': <__main__.RunningMean object at 0x11ec2e850>,
 '_y2s': <__main__.RunningMean object at 0x11ec1ae90>,
 'x_var': 0.6666666666666661,
 'xs': <__main__.RunningMean object at 0x11ec0cc50>,
 'xy_cov': 1.0,
 'y_var': 1.5555555555555571,
 'ys': <__main__.RunningMean object at 0x11ec18d50>}
{'N': 3, 'mean': 2.0, 'q': deque([1, 2, 3])}
{'N': 3, 'mean': 2.333333333333333, 'q': deque([1, 2, 4])}
{'N': 3, 'mean': 4.666666666666666, 'q': deque([1, 4, 9])}
{'N': 3, 'mean': 7.0, 'q': deque([1, 4, 16])}
{'N': 3, 'mean': 5.666666666666666, 'q': deque([1, 4, 12])}
